# Exercise: CPU and the DevCloud

In this exercise, we will load a model on to the Intel Xeon CPU and perform an inference on it. We will then calculate the time it takes to do the same.

<span class="graffiti-highlight graffiti-id_g9b3e7l-id_08m53df"><i></i><button>Graffiti Sample Button (edit me)</button></span>



#### Set up paths so we can run Dev Cloud utilities
You *must* run this every time they enter a Workspace session.

In [ ]:
%env PATH=/opt/conda/bin:/opt/spark-2.4.3-bin-hadoop2.7/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/opt/intel_devcloud_support
import os
import sys
sys.path.insert(0, os.path.abspath('/opt/intel_devcloud_support'))
sys.path.insert(0, os.path.abspath('/opt/intel'))

## The model

We will be using the `vehicle-license-plate-detection-barrier-0106` model for this exercise. Remember that to run a model on the CPU, we need to use FP32 as the model precision.

The model is present in the `/data/models/intel` folder.

We will be running inference on an image of a car. The path to the image is `/data/resources/car.png`

# Step 1: Creating a Python Script

The first step is to create a python script that you can use to load the model and perform an inference. I have used the `writefile` magic to create a python file called `inference_cpu_model.py`. You will need to complete this file.

In [29]:
%%writefile inference_cpu_model.py

import time
import numpy as np
import cv2
from openvino.inference_engine import IENetwork
from openvino.inference_engine import IECore
from openvino.inference_engine import IECore
import argparse

def main(args):
    model=args.model_path
    model_weights=model+'.bin'
    model_structure=model+'.xml'
    
    start=time.time()
    
    print("Starting... Model:", model)
    
    # TODO: Load the model
    plugin = IECore()
    print("Inference Engine plugin created")

    network = IENetwork(model_structure, model_weights)
    print("Network created from spec")

    exec_network = plugin.load_network(network, 'CPU', num_requests=1)
    print("Network loaded into Inference Engine")
    
    print(f"Time taken to load model = {time.time()-start} seconds")
    
    # Reading and Preprocessing Image
    input_img=cv2.imread('/data/resources/car.png')
    input_img=cv2.resize(input_img, (300,300), interpolation = cv2.INTER_AREA)
    input_img=np.moveaxis(input_img, -1, 0)
    print("Input image prepared")

    # TODO: Prepare the model for inference (create input dict etc.)
    input_key = next(iter(network.inputs))
    input_dict = {input_key:input_img}
    print("Input prepared, key:", input_key)
    
    start=time.time()
    for i in range(10):
        # TODO: Run Inference in a Loop
        print("Inference #", i, "start...")
        exec_network.infer(input_dict)
        print("Inference #", i, "finished")
    
    print(f"Time Taken to run 10 Infernce on CPU is = {time.time()-start} seconds")

if __name__=='__main__':
    parser=argparse.ArgumentParser()
    parser.add_argument('--model_path', required=True)
    
    args=parser.parse_args() 
    main(args)

Overwriting inference_cpu_model.py


<span class="graffiti-highlight graffiti-id_6t269sv-id_2g8nwk3"><i></i><button>Show Solution</button></span>

## Step 2: Creating a job submission script

To submit a job to the devcloud, we need to create a script. I have named the script as `inference_cpu_model_job.sh`.

Can you write a script that will take the model path as a command line argument and then call the python file you created in the previous cell with the path to the model?

In [30]:
%%writefile inference_cpu_model_job.sh

exec 1>/output/stdout.log 2>/output/stderr.log
mkdir -p /output

#TODO: Create job submission script
MODELPATH=$1

# Run the load model python script
python3 inference_cpu_model.py --model_path ${MODELPATH}

cd /output
tar zcvf output.tgz stdout.log stderr.log

Overwriting inference_cpu_model_job.sh


<span class="graffiti-highlight graffiti-id_vc779df-id_z9ijl86"><i></i><button>Show Solution</button></span>

## Step 3: Running on the CPU

In the cell below, can you write the qsub command that will submit your job to the Intel CPU?

To get a list of hardware devices available on the DevCloud, you can go to [this link](https://devcloud.intel.com/edge/get_started/devcloud/)

In [31]:
# TODO: Write qsub command
job_id_core = !qsub inference_cpu_model_job.sh -d . -l nodes=1:tank-870:e3-1268l-v5 -F "/data/models/intel/vehicle-license-plate-detection-barrier-0106/FP32/vehicle-license-plate-detection-barrier-0106" -N store_core 
print(job_id_core[0])

bYEM1CkVlr3eSmw23zEH4TCxAPZfdrlw


<span class="graffiti-highlight graffiti-id_qnrfru0-id_7ofr7nk"><i></i><button>Show Solution</button></span>

## Step 4: Getting the Live Stat Values

By running the below command, we can see the live status of the commands.

<span class="graffiti-highlight graffiti-id_9xnofi7-id_m9v5xi8"><i></i><button>Graffiti Sample Button (edit me)</button></span>

In [32]:
import liveQStat
liveQStat.liveQStat()

## Step 5: Get the results

Running the cell below will get the output files from our job

In [33]:
import get_results

get_results.getResults(job_id_core[0], get_stderr=True, filename="output.tgz", blocking=True)

getResults() is blocking until results of the job (id:bYEM1CkVlr3eSmw23zEH4TCxAPZfdrlw) are ready.
Please wait............................................................................Success!
output.tgz was downloaded in the same folder as this notebook.


In [34]:
!tar zxf output.tgz

In [35]:
!cat stdout.log

Starting... Model: /data/models/intel/vehicle-license-plate-detection-barrier-0106/FP32/vehicle-license-plate-detection-barrier-0106
Inference Engine plugin created
Network created from spec
Network loaded into Inference Engine
Time taken to load model = 0.9887981414794922 seconds
Input image prepared
Input prepared, key: Placeholder
Inference # 0 start...
Inference # 0 finished
Inference # 1 start...
Inference # 1 finished
Inference # 2 start...
Inference # 2 finished
Inference # 3 start...
Inference # 3 finished
Inference # 4 start...
Inference # 4 finished
Inference # 5 start...
Inference # 5 finished
Inference # 6 start...
Inference # 6 finished
Inference # 7 start...
Inference # 7 finished
Inference # 8 start...
Inference # 8 finished
Inference # 9 start...
Inference # 9 finished
Time Taken to run 10 Infernce on CPU is = 0.07056117057800293 seconds
